In [26]:
import gensim
import nltk
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
from os import listdir
from os.path import isfile, join
#nltk.download()
import string 
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
import pickle 
import pandas as pd 
import os

print("out")

#case metadata
casedata = pd.read_pickle('casedata.pkl')

#taking only year 2013
yearr="2013"
df=casedata[(casedata["year"] == yearr)]
print(len(df))

#reading documents
yearr="2013"
data=[]
docLabels=[]
for i in range(1,13):
    pathname = os.path.join("Filtered/"+yearr, str(i))
    files=os.listdir(pathname)
    
    for file in files:
        file1=file.split("_")[0]
        caseid=file1
        if caseid not in docLabels:
            
            try:
                data.append(open(pathname+"/"+file).read())
            except:
                print(file)
                print(i)
                continue
            docLabels.append(caseid)


        else:
            print(file)
            print(i)
print(len(docLabels))
print(len(data))

out
481
.DS_Store
1
X1JR2IQ003_contentMajOp_E. GRADY JOLLY.txt
6
X1K7E4U003_contentMajOp_PLAGE R.txt
8
X1K89GO003_contentMajOp_LOURIE.txt
8
481
481


In [31]:



def clean(text):

    text=text.lower()
    table = str.maketrans("", "", string.punctuation)
    wnl = nltk.WordNetLemmatizer()
    
    
    text=text.translate(table)
    #print("without punctuation ")
    #print(text)
    token = nltk.word_tokenize(text)
    #print("tokenized")
    #print(token)
    

    stop_words = set(stopwords.words('english'))
    filtered_sentence = []

    for w in token:
        if w not in stop_words:
            filtered_sentence.append(w)
   
    token=filtered_sentence
    #print("without stop words")
    #print(token)
    token=[wnl.lemmatize(t) for t in token]
    #print("after lemma")
    #print(token)
    return token

new_data=[]
for d in data:
    d=clean(d)
    new_data.append(d)
data=new_data

In [32]:
#create iterator of data and doclabels
class LabeledLineSentence(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list
    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
              yield gensim.models.doc2vec.LabeledSentence(doc,[self.labels_list[idx]])


In [33]:

it = LabeledLineSentence(data, docLabels)


In [34]:
model = gensim.models.Doc2Vec(size=300, min_count=0, alpha=0.025, min_alpha=0.025)
model.build_vocab(it)

#training 
for epoch in range(100):
    #print('iteration '+str(epoch+1))
    model.train(it,total_examples=model.corpus_count,epochs=model.epochs)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

model.save('doc2vec.model')
print("model saved")

/Users/amaladeshpande/anaconda3/lib/python3.6/site-packages/gensim/models/doc2vec.py:366: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/Users/amaladeshpande/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  


model saved


In [35]:
d2v_model = gensim.models.doc2vec.Doc2Vec.load('doc2vec.model')

In [43]:
columns1=['CaseId','Date','Vector']

i=0
docvector= pd.DataFrame(columns=columns1)
for caseid in docLabels:
    
    date=df.loc[caseid]["date"]
    docvec = d2v_model.docvecs[caseid]
    docvector = docvector.append({'CaseId':caseid,'Date':date,'Vector':docvec},ignore_index=True)
docvector

,CaseId,Date,Vector
0,X1F0UMON,1/10/13,"[-1.13301, 1.55332, 0.508589, -1.71668, 0.9915..."
1,X1J3RP6003,1/14/13,"[0.0507614, 1.40233, 2.64493, -0.93347, -1.317..."
2,X1J2U46003,1/4/13,"[-2.77378, -3.81962, 1.00597, -0.987574, 3.688..."
3,X1J62O6003,1/28/13,"[-0.842199, 0.589425, 1.16145, -0.89405, 1.893..."
4,X1EU3KGN,1/17/13,"[1.4258, 2.14917, -1.24492, 1.33474, 4.27253, ..."
5,X1J2NRO003,1/2/13,"[1.13794, -0.335605, 1.06919, -1.46027, 1.3642..."
6,X1Q6MITDN5O2,1/28/13,"[-0.228807, 2.97989, 1.54094, 1.22841, 1.24473..."
7,X1J3JV8003,1/11/13,"[0.343353, 0.477289, 2.51193, -2.08409, 3.1339..."
8,X1J3FQQ003,1/10/13,"[-1.5641, 2.31187, -0.113477, -1.76272, 5.2490..."
9,X1J6PGC003,1/31/13,"[0.370016, 0.438731, 0.538213, 1.56467, 0.7623..."


In [48]:
#finding similar docs
sims = d2v_model.docvecs.most_similar('XVITNB80000N')
print((sims))

[('X1JE6MG003', 0.530697762966156), ('X1J434M003', 0.5251684188842773), ('X1K9LH0003', 0.515308141708374), ('X1GBNA0N', 0.5034132599830627), ('X1JUL6Q003', 0.49790409207344055), ('X1JUEDM003', 0.49229660630226135), ('X1JPBL6003', 0.4922097325325012), ('X1K9600003', 0.49122947454452515), ('X1JLVJA003', 0.4861646294593811), ('X1KU5HS003', 0.4857080578804016)]


In [50]:
docvector.to_pickle("docvector.pkl")